# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [3]:
import pandas as pd
import numpy as np
import math as math

###Importar Excel
data_Treinamento= pd.read_excel('Tweets.xlsx',sheet_name=0)
data_Teste= pd.read_excel('Tweets.xlsx',sheet_name=1)

###Função Limpeza
Treino = data_Treinamento['Treinamento'].str.replace('@', '')
Treino = Treino.str.replace('rt', '')
Treino = Treino.str.replace(',', '')
Treino = Treino.str.replace('.', '')
Treino = Treino.str.replace(':', '')
Treino = Treino.str.replace(';', '')
Treino = Treino.str.replace('\n', '')
Treino = Treino.str.replace('"', '')

Teste = data_Teste['Teste'].str.replace('@', '')
Teste = Teste.str.replace('rt', '')
Teste = Teste.str.replace(',', '')
Teste = Teste.str.replace('.', '')
Teste = Teste.str.replace(':', '')
Teste = Teste.str.replace(';', '')
Teste = Teste.str.replace('\n', '')
Teste = Teste.str.replace('"', '')

data_Treinamento['Treinamento'] = Treino
data_Teste['Teste'] = Teste

###Separando em Relevante e Irrelevante
Re1= data_Treinamento.Treinamento[data_Treinamento.Controle == 1]
Irr1=data_Treinamento.Treinamento[data_Treinamento.Controle == 0]

###Relevancia
Pr = len(Re1)/len(data_Treinamento)
Pirr = len(Irr1)/len(data_Treinamento)

###Separando frases em listas de palavras
Palavras_relevantes=[]

for i in Re1:
    Palavras_relevantes.append(i.split())

Palavras_irrelevantes=[]

for i in Irr1:
    Palavras_irrelevantes.append(i.split())
    
Palavras_total=[]

for i in Treino:
    Palavras_total.append(i.split())
    
###Tirando items repetidos
Laplace= []

for i in Palavras_total:
    for e in i:
        if e.lower() not in Laplace:
            Laplace.append(e.lower())
            
###Probabilidade de uma palavra ser relevante
def Palavra_relevante(palavra):
    soma=1
    total=0
    for i in Palavras_relevantes:
        total+=len(i)
        for e in i:
            if e.lower()==palavra.lower():
                soma+=1
    return soma/(total+len(Laplace))

###Probabilidade de uma palavra ser irrelevante
def Palavra_irrelevante(palavra):
    soma=1
    total=0
    for i in Palavras_irrelevantes:
        total+=len(i)
        for e in i:
            if e.lower()==palavra.lower():
                soma+=1
    return soma/(total+len(Laplace))

###Classificação
def Classificação(lista):
    classificado = []
    for i in lista:         
        palavra = i.split()
        Prelevante= []
        Pirrelevante = []
        for e in palavra:         
            Prelevante.append(Palavra_relevante(e))
            Pirrelevante.append(Palavra_irrelevante(e))
        PR = 1
        PI = 1
        for i in Prelevante:
            PR *= i
        for i in Pirrelevante:
            PI *= i
        PRfrase = PR * Pr 
        PIfrase = PI * Pirr
        if PRfrase > PIfrase:
            classificado.append('Relevante')
        else:
            classificado.append('Irrelevante')
    return classificado    
    
###Analise base Teste    
Frases_teste=[]

for i in Teste:
    Frases_teste.append(i)
    
Final = Classificação(Frases_teste)    
    
print(Final)

['Relevante', 'Relevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Relevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Relevante', 'Irrelevante', 'Relevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Relevante', 'Irrel

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [5]:
teste=data_Teste.Controle

###Positivos falsos
Posfal=0

for i in range(len(Final)) and range(len(teste)):
    if Final[i] == 'Relevante' and teste[i] == 0:
        Posfal+=1
print('Positivos Falsos:')
print(Posfal/200)

###Positivo verdadeiro
Posver=0

for i in range(len(Final)) and range(len(teste)):
    if Final[i] == 'Relevante' and teste[i] == 1:
        Posver+=1
print('Positivos verdadeiro:')
print(Posver/200)

###Negativo verdadeiro
Negver=0

for i in range(len(Final)) and range(len(teste)):
    if Final[i] == 'Irrelevante' and teste[i] == 0:
        Negver+=1
print('Negativo verdadeiro:')
print(Negver/200)

###Negativo falso
Negfal=0

for i in range(len(Final)) and range(len(teste)):
    if Final[i] == 'Irrelevante' and teste[i] == 1:
        Negfal+=1
print('Negativo falso:')
print(Negfal/200)




Positivos Falsos:
0.245
Positivos verdadeiro:
0.065
Negativo verdadeiro:
0.575
Negativo falso:
0.115


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
